In [10]:
import re 

def validate_email(email):
    pattern = r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]'

    if re.match(pattern, email):
        return True

    else:
        return False

email1 = "artem@golubnichij.ru"
email2 = "artem__golubnichij.ru"
email3 = "!rtem@golubnichij.ru"
email4 = "art em@golubnichij.ru"
email5 = "art234234em@golubnichij.ru"


print(validate_email(email1))
print(validate_email(email2))
print(validate_email(email3))
print(validate_email(email4))
print(validate_email(email5))


True
False
False
False
True
